In [1]:
from scipy import stats
from scipy.stats import norm, skew
from sklearn.model_selection import KFold
import os
import sys
import numpy as np
import pandas as pd
import random
from collections import Counter, defaultdict
from sklearn import model_selection
# from sklearn.preprocessing import StandardScaler
all_data = pd.read_csv('../data/forauto.csv')
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [2]:
all_data

,Uid,Uid_count,Category,Subcategory,Concept,Title_len,Title_number,Alltags_len,Alltags_number,img_length,...,followerCount,followingCount,Ispublic,label,img_file,Category.1,Concept.1,Subcategory.1,Alltags,Title
0,21894,4,5,65,75,33,6,128,12,333,...,2405,2729,1,11.18,train/59@N75/775.jpg,Fashion,glam,Fashion,rock punk transgender tranny electronicmusic e...,Luis Drayton - Edinburgh shoot #6
1,53866,13,0,75,139,56,13,800,65,500,...,8491,236,1,15.15,train/1@N18/1075.jpg,Travel&Active&Sports,fifa,Soccer,brazil rio brasil riodejaneiro by maria fifa m...,Arena da Barra - Arena HSBC - Arena do PAN #...
2,26948,1,3,42,480,12,2,188,23,500,...,1550,1346,1,10.99,train/351@N64/4890.jpg,Entertainment,cinema,Movies,old cinema beauty marilyn photoshop movie joke...,MARILYN 2015
3,355,1,10,68,225,18,3,61,9,500,...,1233,1115,1,8.63,train/6@N59/6568.jpg,Holiday&Celebrations,old,Birthday,pictures old family scans brothers sister 1958...,Knikkertijd - 1959
4,315,31,2,43,317,8,1,146,19,500,...,5781,13,1,11.16,train/1617@N40/7079.jpg,Food,thirsty,Drinks,hot sahara animal animals desert bottles drink...,CAMELS01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,2,0,35,301,6,1,78,13,500,...,489,612,1,0.00,test/49686@N77/1118324.jpg,Travel&Active&Sports,sea,Seattle,natur nature frog sea lake fish outdoor macro ...,Quaak!
486190,12280,7,0,60,110,25,4,164,23,500,...,0,0,1,0.00,test/31147@N75/1118329.jpg,Travel&Active&Sports,cars,Cars,11 arisk okruh veternov 2016 10 jn preov veter...,arisk okruh veternov 2016
486191,43496,9,4,28,186,41,7,85,9,500,...,702,113,1,0.00,test/5656@N26/1118336.jpg,Animal,cat,Cats,holland netherlands amsterdam cat nl ramses gu...,Bike & Cat / Eerste Anjeliers Dwarsstraat
486192,5492,17,7,27,54,15,3,305,35,500,...,1092,389,1,0.00,test/28302@N36/1118349.jpg,Whether&Season,white,Snowing,street people bw white black netherlands monoc...,The Stairs crew


In [3]:
all_data.columns

Index(['Uid', 'Uid_count', 'Category', 'Subcategory', 'Concept', 'Title_len',
       'Title_number', 'Alltags_len', 'Alltags_number', 'img_length',
       'img_width', 'pixel', 'img_model', 'svd_mode_t_0', 'svd_mode_t_1',
       'svd_mode_t_2', 'svd_mode_t_3', 'svd_mode_t_4', 'svd_mode_t_5',
       'svd_mode_t_6', 'svd_mode_t_7', 'svd_mode_t_8', 'svd_mode_t_9',
       'svd_mode_t_10', 'svd_mode_t_11', 'svd_mode_t_12', 'svd_mode_t_13',
       'svd_mode_t_14', 'svd_mode_t_15', 'svd_mode_t_16', 'svd_mode_t_17',
       'svd_mode_t_18', 'svd_mode_t_19', 'svd_mode_0', 'svd_mode_1',
       'svd_mode_2', 'svd_mode_3', 'svd_mode_4', 'svd_mode_5', 'svd_mode_6',
       'svd_mode_7', 'svd_mode_8', 'svd_mode_9', 'svd_mode_10', 'svd_mode_11',
       'svd_mode_12', 'svd_mode_13', 'svd_mode_14', 'svd_mode_15',
       'svd_mode_16', 'svd_mode_17', 'svd_mode_18', 'svd_mode_19', 'Mediatype',
       'hour', 'day', 'weekday', 'week_hour', 'year_weekday', 'Longitude',
       'Latitude', 'Geoaccuracy', 'phot

In [4]:
useless_cols = ['Category.1', 'Concept.1', 'Subcategory.1', 'img_file','firstdate','firstweek', 'firstmonth', 
                'firstdatetaken', 'firstdatetakenweek','firstdatetakenmonth']

cate_cols = ['Uid', 'Category', 'Latitude', 'Geoaccuracy', 'Subcategory', 'Concept', 'Mediatype', 
             'hour', 'day', 'weekday', 'week_hour','Geoaccuracy', 'ispro' , 'Ispublic', 'img_model']
# useless_cols += cate_cols
all_data = all_data.drop(useless_cols,axis=1)

columns = ['Title_len', 'Title_number', 'Alltags_len', 'Alltags_number', 
           'photo_count', 'totalTags', 'totalGeotagged', 'totalFaves',
          'totalInGroup','photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']


for i in columns:
    all_data[i] = np.log1p(all_data[i])
    
train = all_data[:-180581]
test = all_data[-180581:]

test = test.reset_index(drop=True)

# y = train[['label']].values
# X = train.drop(['label'],axis=1).values
# df_test = test.drop(['label'],axis=1).values

In [5]:
from dataclasses import dataclass, field
import json
import logging
import os
from typing import Optional

import numpy as np
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoConfig,
    Trainer,
    EvalPrediction,
    set_seed
)
from transformers.training_args import TrainingArguments

# from multimodal_transformers.data import load_data_from_folder
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular

logging.basicConfig(level=logging.INFO)
os.environ['COMET_MODE'] = 'DISABLED'


/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-05 15:55:57.598220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-05 15:55:59.214146: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-05 15:55:59.214292: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerr

In [6]:
def load_data_from_folder(
    folder_path,
    text_cols,
    tokenizer,
    label_col,
    label_list=None,
    categorical_cols=None,
    numerical_cols=None,
    sep_text_token_str=" ",
    categorical_encode_type="none",
    numerical_transformer_method="quantile_normal",
    empty_text_values=None,
    replace_empty_text=None,
    max_token_length=None,
    debug=False,
    debug_dataset_size=100,
):
    """
    Function to load tabular and text data from a specified folder

    Loads train, test and/or validation text and tabular data from specified
    folder path into TorchTextDataset class and does categorical and numerical
    data preprocessing if specified. Inside the folder, there is expected to be
    a train.csv, and test.csv (and if given val.csv) containing the training, testing,
    and validation sets respectively

    Args:
        folder_path (str): The path to the folder containing `train.csv`, and `test.csv` (and if given `val.csv`)
        text_cols (:obj:`list` of :obj:`str`): The column names in the dataset that contain text
            from which we want to load
        tokenizer (:obj:`transformers.tokenization_utils.PreTrainedTokenizer`):
            HuggingFace tokenizer used to tokenize the input texts as specifed by text_cols
        label_col (str): The column name of the label, for classification the column should have
            int values from 0 to n_classes-1 as the label for each class.
            For regression the column can have any numerical value
        label_list (:obj:`list` of :obj:`str`, optional): Used for classification;
            the names of the classes indexed by the values in label_col.
        categorical_cols (:obj:`list` of :obj:`str`, optional): The column names in the dataset that
            contain categorical features. The features can be already prepared numerically, or
            could be preprocessed by the method specified by categorical_encode_type
        numerical_cols (:obj:`list` of :obj:`str`, optional): The column names in the dataset that contain numerical features.
            These columns should contain only numeric values.
        sep_text_token_str (str, optional): The string token that is used to separate between the
            different text columns for a given data example. For Bert for example,
            this could be the [SEP] token.
        categorical_encode_type (str, optional): Given categorical_cols, this specifies
            what method we want to preprocess our categorical features.
            choices: [ 'ohe', 'binary', None]
            see encode_features.CategoricalFeatures for more details
        numerical_transformer_method (str, optional): Given numerical_cols, this specifies
            what method we want to use for normalizing our numerical data.
            choices: ['yeo_johnson', 'box_cox', 'quantile_normal', None]
            see https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html
            for more details
        empty_text_values (:obj:`list` of :obj:`str`, optional): specifies what texts should be considered as
            missing which would be replaced by replace_empty_text
        replace_empty_text (str, optional): The value of the string that will replace the texts
            that match with those in empty_text_values. If this argument is None then
            the text that match with empty_text_values will be skipped
        max_token_length (int, optional): The token length to pad or truncate to on the
            input text
        debug (bool, optional): Whether or not to load a smaller debug version of the dataset

    Returns:
        :obj:`tuple` of `tabular_torch_dataset.TorchTextDataset`:
            This tuple contains the
            training, validation and testing sets. The val dataset is :obj:`None` if
            there is no `val.csv` in folder_path
    """
    train_df = pd.read_csv(join(folder_path, "train.csv"), index_col=0)
    test_df = pd.read_csv(join(folder_path, "test.csv"), index_col=0)
    if exists(join(folder_path, "val.csv")):
        val_df = pd.read_csv(join(folder_path, "val.csv"), index_col=0)
    else:
        val_df = None

    return load_train_val_test_helper(
        train_df,
        val_df,
        test_df,
        text_cols,
        tokenizer,
        label_col,
        label_list,
        categorical_cols,
        numerical_cols,
        sep_text_token_str,
        categorical_encode_type,
        numerical_transformer_method,
        empty_text_values,
        replace_empty_text,
        max_token_length,
        debug,
        debug_dataset_size,
    )

In [7]:
def load_train_val_test_helper(
    train_df,
    val_df,
    test_df,
    text_cols,
    tokenizer,
    label_col,
    label_list=None,
    categorical_cols=None,
    numerical_cols=None,
    sep_text_token_str=" ",
    categorical_encode_type="none",
    numerical_transformer_method="quantile_normal",
    empty_text_values=None,
    replace_empty_text=None,
    max_token_length=None,
    debug=False,
    debug_dataset_size=100,
):
    if categorical_encode_type == "ohe" or categorical_encode_type == "binary":
        dfs = [df for df in [train_df, val_df, test_df] if df is not None]
        data_df = pd.concat(dfs, axis=0)
        cat_feat_processor = CategoricalFeatures(
            data_df, categorical_cols, categorical_encode_type
        )
        vals = cat_feat_processor.fit_transform()
        cat_df = pd.DataFrame(vals, columns=cat_feat_processor.feat_names)
        data_df = pd.concat([data_df, cat_df], axis=1)
        categorical_cols = cat_feat_processor.feat_names

        len_train = len(train_df)
        len_val = len(val_df) if val_df is not None else 0

        train_df = data_df.iloc[:len_train]
        if val_df is not None:
            val_df = data_df.iloc[len_train : len_train + len_val]
            len_train = len_train + len_val
        test_df = data_df.iloc[len_train:]

        categorical_encode_type = None

    if numerical_transformer_method != "none":
        if numerical_transformer_method == "yeo_johnson":
            numerical_transformer = PowerTransformer(method="yeo-johnson")
        elif numerical_transformer_method == "box_cox":
            numerical_transformer = PowerTransformer(method="box-cox")
        elif numerical_transformer_method == "quantile_normal":
            numerical_transformer = QuantileTransformer(output_distribution="normal")
        else:
            raise ValueError(
                f"preprocessing transformer method "
                f"{numerical_transformer_method} not implemented"
            )
        num_feats = load_num_feats(train_df, convert_to_func(numerical_cols))
        numerical_transformer.fit(num_feats)
    else:
        numerical_transformer = None

    train_dataset = load_data(
        train_df,
        text_cols,
        tokenizer,
        label_col,
        label_list,
        categorical_cols,
        numerical_cols,
        sep_text_token_str,
        categorical_encode_type,
        numerical_transformer,
        empty_text_values,
        replace_empty_text,
        max_token_length,
        debug,
        debug_dataset_size,
    )
    test_dataset = load_data(
        test_df,
        text_cols,
        tokenizer,
        label_col,
        label_list,
        categorical_cols,
        numerical_cols,
        sep_text_token_str,
        categorical_encode_type,
        numerical_transformer,
        empty_text_values,
        replace_empty_text,
        max_token_length,
        debug,
        debug_dataset_size,
    )

    if val_df is not None:
        val_dataset = load_data(
            val_df,
            text_cols,
            tokenizer,
            label_col,
            label_list,
            categorical_cols,
            numerical_cols,
            sep_text_token_str,
            categorical_encode_type,
            numerical_transformer,
            empty_text_values,
            replace_empty_text,
            max_token_length,
            debug,
            debug_dataset_size,
        )
    else:
        val_dataset = None

    return train_dataset, val_dataset, test_dataset


In [8]:
def pandas_split_valid_test_dataset(pandas_dataset, valid_ratio=0.2,  shuffle=True):
    
    index = list(range(len(pandas_dataset)))
    
    length = len(pandas_dataset)
    len_valid = int(length * valid_ratio + 0.6)
    # len_test = int(length * test_ratio + 0.6)

    train_data = pandas_dataset.loc[index[:-len_valid]]
    valid_data = pandas_dataset.loc[index[-len_valid:]]
    # test_data = pandas_dataset.loc[index[-len_test:]]
    return train_data, valid_data

train_df, valid_df = pandas_split_valid_test_dataset(train, valid_ratio=0.2, shuffle=True)
print(len(train_df), len(valid_df))

244490 61123


In [9]:
valid_df = valid_df.reset_index(drop=True)

In [11]:
# train_df, val_df = np.split(train.sample(frac=1), [int(.8*len(data_df)), int(.9 * len(data_df))])
# print('Num examples train-val-test')
# print(len(train_df), len(val_df), len(test_df))
# train_df.to_csv('train.csv')
# valid_df.to_csv('val.csv')
# test.to_csv('test.csv')

In [10]:
@dataclass
class ModelArguments:
  # """
  # Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
  # """

    model_name_or_path: str = field(
      metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
  )
    config_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
  )
    tokenizer_name: Optional[str] = field(
      default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
  )
    cache_dir: Optional[str] = field(
      default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
  )


@dataclass
class MultimodalDataTrainingArguments:
  # """
  # Arguments pertaining to how we combine tabular features
  # Using `HfArgumentParser` we can turn this class
  # into argparse arguments to be able to specify them on
  # the command line.
  # """

    data_path: str = field(metadata={
                            'help': 'the path to the csv file containing the dataset'
                        })
    column_info_path: str = field(
      default=None,
      metadata={
          'help': 'the path to the json file detailing which columns are text, categorical, numerical, and the label'
      })

    column_info: dict = field(
      default=None,
      metadata={
          'help': 'a dict referencing the text, categorical, numerical, and label columns'
                  'its keys are text_cols, num_cols, cat_cols, and label_col'
      })

    categorical_encode_type: str = field(default='none',
                                        metadata={
                                            'help': 'sklearn encoder to use for categorical data',
                                            # 'choices': ['ohe', 'binary', 'label', 'none']
                                            'choices':['none']
                                        })
    numerical_transformer_method: str = field(default='yeo_johnson',
                                            metadata={
                                                'help': 'sklearn numerical transformer to preprocess numerical data',
                                                'choices': ['yeo_johnson', 'box_cox', 'quantile_normal', 'none']
                                            })
    task: str = field(default="regression",
                    metadata={
                        "help": "The downstream training task",
                        "choices": ["classification", "regression"]
                    })

    mlp_division: int = field(default=16,
                            metadata={
                                'help': 'the ratio of the number of '
                                        'hidden dims in a current layer to the next MLP layer'
                            })
    combine_feat_method: str = field(default='individual_mlps_on_cat_and_numerical_feats_then_concat',
                                    metadata={
                                        'help': 'method to combine categorical and numerical features, '
                                                'see README for all the method'
                                    })
    mlp_dropout: float = field(default=0.1,
                              metadata={
                                'help': 'dropout ratio used for MLP layers'
                              })
    numerical_bn: bool = field(default=True,
                              metadata={
                                  'help': 'whether to use batchnorm on numerical features'
                              })
    use_simple_classifier: str = field(default=True,
                                      metadata={
                                          'help': 'whether to use single layer or MLP as final classifier'
                                      })
    mlp_act: str = field(default='relu',
                        metadata={
                            'help': 'the activation function to use for finetuning layers',
                            'choices': ['relu', 'prelu', 'sigmoid', 'tanh', 'linear']
                        })
    gating_beta: float = field(default=0.2,
                              metadata={
                                  'help': "the beta hyperparameters used for gating tabular data "
                                          "see https://www.aclweb.org/anthology/2020.acl-main.214.pdf"
                              })

    def __post_init__(self):
        assert self.column_info != self.column_info_path
        if self.column_info is None and self.column_info_path:
            with open(self.column_info_path, 'r') as f:
                self.column_info = json.load(f)

In [11]:
text_cols = ['Title', 'Alltags']
# cat_cols = ['Clothing ID', 'Division Name', 'Department Name', 'Class Name']
cat_cols = ['Uid', 'Category', 'Latitude', 'Geoaccuracy', 'Subcategory', 'Concept', 'Mediatype', 
             'hour', 'day', 'weekday', 'week_hour','Geoaccuracy', 'ispro' , 'Ispublic', 'img_model']
label_cols = ['label']
useful_cols = text_cols + cat_cols + label_cols
numerical_cols = [c for c in all_data.columns if c not in useful_cols] 

column_info_dict = {
    'text_cols': text_cols,
    'num_cols': numerical_cols,
    'cat_cols': cat_cols,
    'label_col': 'label'
    # 'label_list': ['Not Recommended', 'Recommended']
}


model_args = ModelArguments(
    model_name_or_path='bert-base-uncased'
)

data_args = MultimodalDataTrainingArguments(
    data_path='.',
    combine_feat_method='gating_on_cat_and_num_feats_then_sum',
    column_info=column_info_dict,
    task='regression'
)

training_args = TrainingArguments(
    output_dir="./logs/model_name",
    logging_dir="./logs/runs",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=256,
    num_train_epochs=3,
    # evaluate_during_training=True,
    logging_steps=25,
    eval_steps=250
)

set_seed(training_args.seed)

In [12]:
numerical_cols

['Uid_count',
 'Title_len',
 'Title_number',
 'Alltags_len',
 'Alltags_number',
 'img_length',
 'img_width',
 'pixel',
 'svd_mode_t_0',
 'svd_mode_t_1',
 'svd_mode_t_2',
 'svd_mode_t_3',
 'svd_mode_t_4',
 'svd_mode_t_5',
 'svd_mode_t_6',
 'svd_mode_t_7',
 'svd_mode_t_8',
 'svd_mode_t_9',
 'svd_mode_t_10',
 'svd_mode_t_11',
 'svd_mode_t_12',
 'svd_mode_t_13',
 'svd_mode_t_14',
 'svd_mode_t_15',
 'svd_mode_t_16',
 'svd_mode_t_17',
 'svd_mode_t_18',
 'svd_mode_t_19',
 'svd_mode_0',
 'svd_mode_1',
 'svd_mode_2',
 'svd_mode_3',
 'svd_mode_4',
 'svd_mode_5',
 'svd_mode_6',
 'svd_mode_7',
 'svd_mode_8',
 'svd_mode_9',
 'svd_mode_10',
 'svd_mode_11',
 'svd_mode_12',
 'svd_mode_13',
 'svd_mode_14',
 'svd_mode_15',
 'svd_mode_16',
 'svd_mode_17',
 'svd_mode_18',
 'svd_mode_19',
 'year_weekday',
 'Longitude',
 'photo_count',
 'totalViews',
 'totalTags',
 'totalGeotagged',
 'totalFaves',
 'totalInGroup',
 'photoCount',
 'meanView',
 'meanTags',
 'meanFaves',
 'followerCount',
 'followingCount']

In [14]:
# training_args

In [15]:
tokenizer_path_or_name = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
print('Specified tokenizer: ', tokenizer_path_or_name)
tokenizer = AutoTokenizer.from_pretrained(
    tokenizer_path_or_name,
    cache_dir=model_args.cache_dir,
)

Specified tokenizer:  bert-base-uncased


In [18]:
# model_args

ModelArguments(model_name_or_path='bert-base-uncased', config_name=None, tokenizer_name=None, cache_dir=None)

In [16]:
def load_data(
    data_df,
    text_cols,
    tokenizer,
    label_col,
    label_list=None,
    categorical_cols=None,
    numerical_cols=None,
    sep_text_token_str=" ",
    categorical_encode_type="none",
    numerical_transformer=None,
    empty_text_values=None,
    replace_empty_text=None,
    max_token_length=None,
    debug=False,
    debug_dataset_size=100,
):
    """Function to load a single dataset given a pandas DataFrame

    Given a DataFrame, this function loads the data to a :obj:`torch_dataset.TorchTextDataset`
    object which can be used in a :obj:`torch.utils.data.DataLoader`.

    Args:
        data_df (:obj:`pd.DataFrame`): The DataFrame to convert to a TorchTextDataset
        text_cols (:obj:`list` of :obj:`str`): the column names in the dataset that contain text
            from which we want to load
        tokenizer (:obj:`transformers.tokenization_utils.PreTrainedTokenizer`):
            HuggingFace tokenizer used to tokenize the input texts as specifed by text_cols
        label_col (str): The column name of the label, for classification the column should have
            int values from 0 to n_classes-1 as the label for each class.
            For regression the column can have any numerical value
        label_list (:obj:`list` of :obj:`str`, optional): Used for classification;
            the names of the classes indexed by the values in label_col.
        categorical_cols (:obj:`list` of :obj:`str`, optional): The column names in the dataset that
            contain categorical features. The features can be already prepared numerically, or
            could be preprocessed by the method specified by categorical_encode_type
        numerical_cols (:obj:`list` of :obj:`str`, optional): The column names in the dataset that contain numerical features.
            These columns should contain only numeric values.
        sep_text_token_str (str, optional): The string token that is used to separate between the
            different text columns for a given data example. For Bert for example,
            this could be the [SEP] token.
        categorical_encode_type (str, optional): Given categorical_cols, this specifies
            what method we want to preprocess our categorical features.
            choices: [ 'ohe', 'binary', None]
            see encode_features.CategoricalFeatures for more details
        numerical_transformer (:obj:`sklearn.base.TransformerMixin`): The sklearn numeric
            transformer instance to transform our numerical features
        empty_text_values (:obj:`list` of :obj:`str`, optional): Specifies what texts should be considered as
            missing which would be replaced by replace_empty_text
        replace_empty_text (str, optional): The value of the string that will replace the texts
            that match with those in empty_text_values. If this argument is None then
            the text that match with empty_text_values will be skipped
        max_token_length (int, optional): The token length to pad or truncate to on the
            input text
        debug (bool, optional): Whether or not to load a smaller debug version of the dataset

    Returns:
        :obj:`tabular_torch_dataset.TorchTextDataset`: The converted dataset
    """
    if debug:
        data_df = data_df[:debug_dataset_size]
    if empty_text_values is None:
        empty_text_values = ["nan", "None"]

    text_cols_func = convert_to_func(text_cols)
    categorical_cols_func = convert_to_func(categorical_cols)
    numerical_cols_func = convert_to_func(numerical_cols)

    categorical_feats, numerical_feats = load_cat_and_num_feats(
        data_df, categorical_cols_func, numerical_cols_func, categorical_encode_type
    )
    numerical_feats = normalize_numerical_feats(numerical_feats, numerical_transformer)
    agg_func = partial(agg_text_columns_func, empty_text_values, replace_empty_text)
    texts_cols = get_matching_cols(data_df, text_cols_func)
    logger.info(f"Text columns: {texts_cols}")
    texts_list = data_df[texts_cols].agg(agg_func, axis=1).tolist()
    for i, text in enumerate(texts_list):
        texts_list[i] = f" {sep_text_token_str} ".join(text)
    logger.info(f"Raw text example: {texts_list[0]}")
    hf_model_text_input = tokenizer(
        texts_list, padding=True, truncation=True, max_length=max_token_length
    )
    tokenized_text_ex = " ".join(
        tokenizer.convert_ids_to_tokens(hf_model_text_input["input_ids"][0])
    )
    logger.debug(f"Tokenized text example: {tokenized_text_ex}")
    labels = data_df[label_col].values

    return TorchTabularTextDataset(
        hf_model_text_input,
        categorical_feats,
        numerical_feats,
        labels,
        data_df,
        label_list,
    )

In [18]:
# Get Datasets
from os.path import join, exists
from functools import partial
import logging
from os.path import join, exists

import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import PowerTransformer, QuantileTransformer
from multimodal_transformers.data.data_utils import *
from multimodal_transformers.data.tabular_torch_dataset import TorchTabularTextDataset
train_dataset, val_dataset, test_dataset = load_data_from_folder(
    data_args.data_path,
    data_args.column_info['text_cols'],
    tokenizer,
    label_col=data_args.column_info['label_col'],
    # label_list=data_args.column_info['label_list'],
    categorical_cols=data_args.column_info['cat_cols'],
    numerical_cols=data_args.column_info['num_cols'],
    sep_text_token_str=tokenizer.sep_token,
)

INFO:multimodal_transformers.data.data_utils:62 numerical columns
INFO:multimodal_transformers.data.data_utils:14 categorical columns
INFO:multimodal_transformers.data.data_utils:62 numerical columns
INFO:multimodal_transformers.data.data_utils:Text columns: ['Alltags', 'Title']
INFO:multimodal_transformers.data.data_utils:Raw text example: rock punk transgender tranny electronicmusic electro glam electronica luisdrayton fusionrecords thefusionnetwork lmwcphotography [SEP] Luis Drayton - Edinburgh shoot #6
INFO:multimodal_transformers.data.data_utils:14 categorical columns
INFO:multimodal_transformers.data.data_utils:62 numerical columns
INFO:multimodal_transformers.data.data_utils:Text columns: ['Alltags', 'Title']
INFO:multimodal_transformers.data.data_utils:Raw text example: vintage toys cool nikon peace lego free retro hippie hugs minifig mai68 serie7 d3100 designholic [SEP] May 1968
INFO:multimodal_transformers.data.data_utils:14 categorical columns
INFO:multimodal_transformers.da

In [ ]:
# num_labels = len(np.unique(train_dataset.labels))
# num_labels

In [37]:
config = AutoConfig.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )
tabular_config = TabularConfig(num_labels=1,
                               cat_feat_dim=train_dataset.cat_feats.shape[1],
                               numerical_feat_dim=train_dataset.numerical_feats.shape[1],
                               **vars(data_args))
config.tabular_config = tabular_config

In [38]:
model = AutoModelWithTabular.from_pretrained(
        model_args.config_name if model_args.config_name else model_args.model_name_or_path,
        config=config,
        cache_dir=model_args.cache_dir
    )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertWithTabular: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertWithTabular from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertWithTabular from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertWithTabular were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['tabular_

In [ ]:
# import numpy as np
# from scipy.special import softmax
# from sklearn.metrics import (
#     auc,
#     precision_recall_curve,
#     roc_auc_score,
#     f1_score,
#     confusion_matrix,
#     matthews_corrcoef,
# )
import math
import numpy as np
from sklearn.metrics import (
    auc,
    precision_recall_curve,
    roc_auc_score,
    f1_score,
    confusion_matrix,
    matthews_corrcoef,
    mean_squared_error,
    mean_absolute_error,
)



def calc_regression_metrics(preds, labels):
    mse = mean_squared_error(labels, preds)
    rmse = math.sqrt(mse)
    mae = mean_absolute_error(labels, preds)
    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
    }

# def calc_classification_metrics(p: EvalPrediction):
#     predictions = p.predictions[0]
#     pred_labels = np.argmax(predictions, axis=1)
#     pred_scores = softmax(predictions, axis=1)[:, 1]
#     labels = p.label_ids
#     if len(np.unique(labels)) == 2:  # binary classification
#         roc_auc_pred_score = roc_auc_score(labels, pred_scores)
#         precisions, recalls, thresholds = precision_recall_curve(labels,
#                                                                 pred_scores)
#         fscore = (2 * precisions * recalls) / (precisions + recalls)
#         fscore[np.isnan(fscore)] = 0
#         ix = np.argmax(fscore)
#         threshold = thresholds[ix].item()
#         pr_auc = auc(recalls, precisions)
#         tn, fp, fn, tp = confusion_matrix(labels, pred_labels, labels=[0, 1]).ravel()
#         result = {'roc_auc': roc_auc_pred_score,
#                 'threshold': threshold,
#                 'pr_auc': pr_auc,
#                 'recall': recalls[ix].item(),
#                 'precision': precisions[ix].item(), 'f1': fscore[ix].item(),
#                 'tn': tn.item(), 'fp': fp.item(), 'fn': fn.item(), 'tp': tp.item()
#                 }
#     else:
#         acc = (pred_labels == labels).mean()
#         f1 = f1_score(y_true=labels, y_pred=pred_labels)
#         result = {
#           "acc": acc,
#           "f1": f1,
#           "acc_and_f1": (acc + f1) / 2,
#           "mcc": matthews_corrcoef(labels, pred_labels)
#       }

#     return result

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=calc_classification_metrics,
)

In [ ]:
%%time
trainer.train()

In [ ]:
import logging
import os
from statistics import mean, stdev
import sys
from typing import Callable, Dict

import numpy as np
from pprint import pformat
from scipy.special import softmax
import torch
from transformers import (
    AutoTokenizer,
    AutoConfig,
    HfArgumentParser,
    Trainer,
    EvalPrediction,
    set_seed,
)

from multimodal_exp_args import (
    MultimodalDataTrainingArguments,
    ModelArguments,
    OurTrainingArguments,
)
from evaluation import calc_classification_metrics, calc_regression_metrics
# from multimodal_transformers.data import load_data_from_folder, load_data_into_folds
from multimodal_transformers.model import TabularConfig
from multimodal_transformers.model import AutoModelWithTabular
from util import create_dir_if_not_exists, get_args_info_as_str

os.environ["COMET_MODE"] = "DISABLED"
logger = logging.getLogger(__name__)


def main():
    parser = HfArgumentParser(
        (ModelArguments, MultimodalDataTrainingArguments, OurTrainingArguments)
    )
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(
            json_file=os.path.abspath(sys.argv[1])
        )
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    if (
        os.path.exists(training_args.output_dir)
        and os.listdir(training_args.output_dir)
        and training_args.do_train
        and not training_args.overwrite_output_dir
    ):
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
        )

    # Setup logging
    create_dir_if_not_exists(training_args.output_dir)
    stream_handler = logging.StreamHandler(sys.stderr)
    file_handler = logging.FileHandler(
        filename=os.path.join(training_args.output_dir, "train_log.txt"), mode="w+"
    )
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
        datefmt="%m/%d/%Y %H:%M:%S",
        handlers=[stream_handler, file_handler],
    )

    logger.info(f"======== Model Args ========\n{get_args_info_as_str(model_args)}\n")
    logger.info(f"======== Data Args ========\n{get_args_info_as_str(data_args)}\n")
    logger.info(
        f"======== Training Args ========\n{get_args_info_as_str(training_args)}\n"
    )

    tokenizer = AutoTokenizer.from_pretrained(
        model_args.tokenizer_name
        if model_args.tokenizer_name
        else model_args.model_name_or_path,
        cache_dir=model_args.cache_dir,
    )

    if not data_args.create_folds:
        train_dataset, val_dataset, test_dataset = load_data_from_folder(
            data_args.data_path,
            data_args.column_info["text_cols"],
            tokenizer,
            label_col=data_args.column_info["label_col"],
            label_list=data_args.column_info["label_list"],
            categorical_cols=data_args.column_info["cat_cols"],
            numerical_cols=data_args.column_info["num_cols"],
            categorical_encode_type=data_args.categorical_encode_type,
            numerical_transformer_method=data_args.numerical_transformer_method,
            sep_text_token_str=tokenizer.sep_token
            if not data_args.column_info["text_col_sep_token"]
            else data_args.column_info["text_col_sep_token"],
            max_token_length=training_args.max_token_length,
            debug=training_args.debug_dataset,
            debug_dataset_size=training_args.debug_dataset_size,
        )
        train_datasets = [train_dataset]
        val_datasets = [val_dataset]
        test_datasets = [test_dataset]
    else:
        train_datasets, val_datasets, test_datasets = load_data_into_folds(
            data_args.data_path,
            data_args.num_folds,
            data_args.validation_ratio,
            data_args.column_info["text_cols"],
            tokenizer,
            label_col=data_args.column_info["label_col"],
            label_list=data_args.column_info["label_list"],
            categorical_cols=data_args.column_info["cat_cols"],
            numerical_cols=data_args.column_info["num_cols"],
            categorical_encode_type=data_args.categorical_encode_type,
            numerical_transformer_method=data_args.numerical_transformer_method,
            sep_text_token_str=tokenizer.sep_token
            if not data_args.column_info["text_col_sep_token"]
            else data_args.column_info["text_col_sep_token"],
            max_token_length=training_args.max_token_length,
            debug=training_args.debug_dataset,
            debug_dataset_size=training_args.debug_dataset_size,
        )
    train_dataset = train_datasets[0]

    set_seed(training_args.seed)
    task = data_args.task
    if task == "regression":
        num_labels = 1
    else:
        num_labels = (
            len(np.unique(train_dataset.labels))
            if data_args.num_classes == -1
            else data_args.num_classes
        )

    def build_compute_metrics_fn(task_name: str) -> Callable[[EvalPrediction], Dict]:
        def compute_metrics_fn(p: EvalPrediction):
            # p.predictions is now a list of objects
            # The first entry is the actual predictions
            predictions = p.predictions[0]
            if task_name == "classification":
                preds_labels = np.argmax(predictions, axis=1)
                if predictions.shape[-1] == 2:
                    pred_scores = softmax(predictions, axis=1)[:, 1]
                else:
                    pred_scores = softmax(predictions, axis=1)
                return calc_classification_metrics(
                    pred_scores, preds_labels, p.label_ids
                )
            elif task_name == "regression":
                preds = np.squeeze(predictions)
                return calc_regression_metrics(preds, p.label_ids)
            else:
                return {}

        return compute_metrics_fn

    total_results = []
    for i, (train_dataset, val_dataset, test_dataset) in enumerate(
        zip(train_datasets, val_datasets, test_datasets)
    ):
        logger.info(f"======== Fold {i+1} ========")
        config = AutoConfig.from_pretrained(
            model_args.config_name
            if model_args.config_name
            else model_args.model_name_or_path,
            cache_dir=model_args.cache_dir,
        )
        tabular_config = TabularConfig(
            num_labels=num_labels,
            cat_feat_dim=train_dataset.cat_feats.shape[1]
            if train_dataset.cat_feats is not None
            else 0,
            numerical_feat_dim=train_dataset.numerical_feats.shape[1]
            if train_dataset.numerical_feats is not None
            else 0,
            **vars(data_args),
        )
        config.tabular_config = tabular_config

        model = AutoModelWithTabular.from_pretrained(
            model_args.config_name
            if model_args.config_name
            else model_args.model_name_or_path,
            config=config,
            cache_dir=model_args.cache_dir,
        )
        if i == 0:
            logger.info(tabular_config)
            logger.info(model)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=build_compute_metrics_fn(task),
        )
        if training_args.do_train:
            trainer.train(
                model_path=model_args.model_name_or_path
                if os.path.isdir(model_args.model_name_or_path)
                else None
            )
            trainer.save_model()

        # Evaluation
        eval_results = {}
        if training_args.do_eval:
            logger.info("*** Evaluate ***")
            eval_result = trainer.evaluate(eval_dataset=val_dataset)
            logger.info(pformat(eval_result, indent=4))

            output_eval_file = os.path.join(
                training_args.output_dir, f"eval_metric_results_{task}_fold_{i+1}.txt"
            )
            if trainer.is_world_process_zero():
                with open(output_eval_file, "w") as writer:
                    logger.info("***** Eval results {} *****".format(task))
                    for key, value in eval_result.items():
                        logger.info("  %s = %s", key, value)
                        writer.write("%s = %s\n" % (key, value))

            eval_results.update(eval_result)

        if training_args.do_predict:
            logging.info("*** Test ***")

            predictions = trainer.predict(test_dataset=test_dataset).predictions[0]
            output_test_file = os.path.join(
                training_args.output_dir, f"test_results_{task}_fold_{i+1}.txt"
            )
            eval_result = trainer.evaluate(eval_dataset=test_dataset)
            logger.info(pformat(eval_result, indent=4))
            if trainer.is_world_process_zero():
                with open(output_test_file, "w") as writer:
                    logger.info("***** Test results {} *****".format(task))
                    writer.write("index\tprediction\n")
                    if task == "classification":
                        predictions = np.argmax(predictions, axis=1)
                    for index, item in enumerate(predictions):
                        if task == "regression":
                            writer.write(
                                "%d\t%3.3f\t%d\n"
                                % (index, item, test_dataset.labels[index])
                            )
                        else:
                            item = test_dataset.get_labels()[item]
                            writer.write("%d\t%s\n" % (index, item))
                output_test_file = os.path.join(
                    training_args.output_dir,
                    f"test_metric_results_{task}_fold_{i+1}.txt",
                )
                with open(output_test_file, "w") as writer:
                    logger.info("***** Test results {} *****".format(task))
                    for key, value in eval_result.items():
                        logger.info("  %s = %s", key, value)
                        writer.write("%s = %s\n" % (key, value))
                eval_results.update(eval_result)
        del model
        del config
        del tabular_config
        del trainer
        torch.cuda.empty_cache()
        total_results.append(eval_results)
    aggr_res = aggregate_results(total_results)
    logger.info("========= Aggr Results ========")
    logger.info(pformat(aggr_res, indent=4))

    output_aggre_test_file = os.path.join(
        training_args.output_dir, f"all_test_metric_results_{task}.txt"
    )
    with open(output_aggre_test_file, "w") as writer:
        logger.info("***** Aggr results {} *****".format(task))
        for key, value in aggr_res.items():
            logger.info("  %s = %s", key, value)
            writer.write("%s = %s\n" % (key, value))


def aggregate_results(total_test_results):
    metric_keys = list(total_test_results[0].keys())
    aggr_results = dict()

    for metric_name in metric_keys:
        if type(total_test_results[0][metric_name]) is str:
            continue
        res_list = []
        for results in total_test_results:
            res_list.append(results[metric_name])
        if len(res_list) == 1:
            metric_avg = res_list[0]
            metric_stdev = 0
        else:
            metric_avg = mean(res_list)
            metric_stdev = stdev(res_list)

        aggr_results[metric_name + "_mean"] = metric_avg
        aggr_results[metric_name + "_stdev"] = metric_stdev
    return aggr_results


if __name__ == "__main__":
    main()